In [1]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [2]:
import tensorflow as tf
from tensorflow.keras import models, layers, regularizers
from tensorflow.keras import backend as K
import cv2
import numpy as np 
import os
import skimage.io as io
import skimage.transform as trans
import numpy as np
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras import backend as keras
import tensorflow as tf

In [3]:
def conv_block(x, filter_size, size, dropout, batch_norm=False):
    
    conv = layers.Conv2D(size, (filter_size, filter_size), padding="same",dilation_rate=2, kernel_initializer = 'he_normal')(x)
    if batch_norm is True:
        conv = layers.BatchNormalization(axis=3)(conv)
    conv = layers.Activation("relu")(conv)

    conv = layers.Conv2D(size, (filter_size, filter_size), padding="same",dilation_rate=2, kernel_initializer = 'he_normal')(conv)
    if batch_norm is True:
        conv = layers.BatchNormalization(axis=3)(conv)
    conv = layers.Activation("relu")(conv)
    
    if dropout > 0:
        conv = layers.Dropout(dropout)(conv)

    return conv


In [5]:
def conv_block_ritnet(x, filter_size, size, dropout, batch_norm=False):
    
    conv = layers.Conv2D(size, (filter_size, filter_size), padding="same")(x)
    if batch_norm is True:
        conv = layers.BatchNormalization(axis=3)(conv)
    conv = layers.Activation("relu")(conv)

    conv = layers.Conv2D(size, (filter_size, filter_size), padding="same")(conv)
    if batch_norm is True:
        conv = layers.BatchNormalization(axis=3)(conv)
    conv = layers.Activation("relu")(conv)
    
    conv = layers.Conv2D(size, (filter_size, filter_size), padding="same")(x)
    if batch_norm is True:
        conv = layers.BatchNormalization(axis=3)(conv)
    conv = layers.Activation("relu")(conv)
    
    conv = layers.Conv2D(size, (filter_size, filter_size), padding="same")(x)
    if batch_norm is True:
        conv = layers.BatchNormalization(axis=3)(conv)
    conv = layers.Activation("relu")(conv)
    
    conv = layers.Conv2D(size, (filter_size, filter_size), padding="same")(x)
    if batch_norm is True:
        conv = layers.BatchNormalization(axis=3)(conv)
    conv = layers.Activation("relu")(conv)
    
    if dropout > 0:
        conv = layers.Dropout(dropout)(conv)

    return conv


In [6]:
def UNet(input_shape, inputs, NUM_CLASSES=1, dropout_rate=0.0, batch_norm=True, second=False, conv_1=[]):
    '''
    UNet, 
    
    '''
    # network structure
    FILTER_NUM = 64 # number of filters for the first layer
    FILTER_SIZE = 3 # size of the convolutional filter
    UP_SAMP_SIZE = 2 # size of upsampling filters
    

#     inputs = layers.Input(input_shape, dtype=tf.float32)

    # Downsampling layers
    # DownRes 1, convolution + pooling
    conv_128 = conv_block(inputs, FILTER_SIZE, FILTER_NUM, dropout_rate, batch_norm)
    pool_64 = layers.MaxPooling2D(pool_size=(2,2))(conv_128)
    # DownRes 2
    conv_64 = conv_block(pool_64, FILTER_SIZE, 2*FILTER_NUM, dropout_rate, batch_norm)
    pool_32 = layers.MaxPooling2D(pool_size=(2,2))(conv_64)
    # DownRes 3
    conv_32 = conv_block(pool_32, FILTER_SIZE, 4*FILTER_NUM, dropout_rate, batch_norm)
    pool_16 = layers.MaxPooling2D(pool_size=(2,2))(conv_32)
    # DownRes 4
    conv_16 = conv_block(pool_16, FILTER_SIZE, 8*FILTER_NUM, dropout_rate, batch_norm)
    pool_8 = layers.MaxPooling2D(pool_size=(2,2))(conv_16)
    # DownRes 5, convolution only
    conv_8 = conv_block(pool_8, FILTER_SIZE, 16*FILTER_NUM, dropout_rate, batch_norm)

    # Upsampling layers
   
    up_16 = layers.UpSampling2D(size=(UP_SAMP_SIZE, UP_SAMP_SIZE), data_format="channels_last")(conv_8)
    if second:
        up_16 = layers.concatenate([up_16, conv_16, conv_1[3]], axis=3)
    else:
        up_16 = layers.concatenate([up_16, conv_16], axis=3)
    up_conv_16 = conv_block(up_16, FILTER_SIZE, 8*FILTER_NUM, dropout_rate, batch_norm)
    # UpRes 7
    
    up_32 = layers.UpSampling2D(size=(UP_SAMP_SIZE, UP_SAMP_SIZE), data_format="channels_last")(up_conv_16)
    if second:
        up_32 = layers.concatenate([up_32, conv_32, conv_1[2]], axis=3)
    else:
        up_32 = layers.concatenate([up_32, conv_32], axis=3)
    up_conv_32 = conv_block(up_32, FILTER_SIZE, 4*FILTER_NUM, dropout_rate, batch_norm)
    # UpRes 8
    
    up_64 = layers.UpSampling2D(size=(UP_SAMP_SIZE, UP_SAMP_SIZE), data_format="channels_last")(up_conv_32)
    if second:
        up_64 = layers.concatenate([up_64, conv_64, conv_1[1]], axis=3)
    else:
        up_64 = layers.concatenate([up_64, conv_64], axis=3)
    up_conv_64 = conv_block(up_64, FILTER_SIZE, 2*FILTER_NUM, dropout_rate, batch_norm)
    # UpRes 9
   
    up_128 = layers.UpSampling2D(size=(UP_SAMP_SIZE, UP_SAMP_SIZE), data_format="channels_last")(up_conv_64)
    if second:
        up_128 = layers.concatenate([up_128, conv_128,conv_1[0]], axis=3)
    else:
        up_128 = layers.concatenate([up_128, conv_128], axis=3)
    up_conv_128 = conv_block(up_128, FILTER_SIZE, FILTER_NUM, dropout_rate, batch_norm)

    # 1*1 convolutional layers
   
    conv_final = layers.Conv2D(NUM_CLASSES, kernel_size=(1,1))(up_conv_128)
    conv_final = layers.BatchNormalization(axis=3)(conv_final)
    conv_final = layers.Activation('sigmoid')(conv_final)  #Change to softmax for multichannel
    
    
    return conv_final,[conv_128, conv_64, conv_32, conv_16]


In [7]:
def double_Unet(input_shape, NUM_CLASSES=1, dropout_rate=0.0, batch_norm=True):
    inputs1 = layers.Input(input_shape, dtype=tf.float32)
    output1, pre_conv = UNet(input_shape, inputs1, NUM_CLASSES=1, dropout_rate=0.2, batch_norm=True)
    inputs2 = inputs1 * output1
    output2,_ =  UNet(input_shape, inputs2, NUM_CLASSES=1, dropout_rate=0.2, 
                      batch_norm=True, second=True, conv_1=pre_conv)
    conv_final = layers.concatenate([output1, output2], axis=3)
    # 1*1 convolutional layers
   
    conv_final = layers.Conv2D(NUM_CLASSES, kernel_size=(1,1))(conv_final)
    conv_final = layers.BatchNormalization(axis=3)(conv_final)
    conv_final = layers.Activation('sigmoid')(conv_final)  #Change to softmax for multichannel
    
    model = models.Model(inputs1, conv_final, name="DoubleUNet")
    return model
    


In [28]:
input_shape = (128,128,3)
model = double_Unet(input_shape, NUM_CLASSES=1, dropout_rate=0.2, batch_norm=True)

In [29]:
model.summary()

Model: "DoubleUNet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
conv2d_39 (Conv2D)              (None, 128, 128, 64) 1792        input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_39 (BatchNo (None, 128, 128, 64) 256         conv2d_39[0][0]                  
__________________________________________________________________________________________________
activation_39 (Activation)      (None, 128, 128, 64) 0           batch_normalization_39[0][0]     
_________________________________________________________________________________________

In [30]:
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        img = cv2.resize(img, (128,128),interpolation = cv2.INTER_NEAREST)
        if img is not None:
            images.append(img)
    return images

In [31]:
import matplotlib.pyplot as plt 
from tensorflow.python.keras import losses

In [32]:
folder= "C:/Users/anark/Downloads/seg_data/train_images"
images = load_images_from_folder(folder)
image_array = np.array(images)

In [33]:
folder= "C:/Users/anark/Downloads/seg_data/train_masks"
masks = load_images_from_folder(folder)
mask_array = np.array(masks)[:,:,:,0]

In [34]:
img = image_array/255

img.shape

mas = mask_array/255

mas[mas > 0.5] = 1
mas[mas <= 0.5] = 0
mas.shape

(2500, 128, 128)

In [35]:
image_array.shape, mask_array.shape

((2500, 128, 128, 3), (2500, 128, 128))

In [36]:
# Split Dataset
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(img, mas, test_size=0.15, random_state=1)

In [37]:
X_train = X_train.astype(np.float16)
X_val = X_val.astype(np.float16)
y_train = y_train.astype(np.float16)
y_val = y_val.astype(np.float16)

In [38]:
X_train.dtype, X_val.dtype, y_train.dtype, y_val.dtype

(dtype('float16'), dtype('float16'), dtype('float16'), dtype('float16'))

In [39]:
X_train.shape,y_train.shape

((2125, 128, 128, 3), (2125, 128, 128))

In [40]:
np.unique(y_val),np.unique(y_train)

(array([0., 1.], dtype=float16), array([0., 1.], dtype=float16))

In [41]:
np.max(X_val),np.max(X_train),np.max(y_val),np.max(y_train)

(1.0, 1.0, 1.0, 1.0)

In [42]:
def dice_coeff(y_true, y_pred):
    smooth = 1.
    # Flatten
    y_true_f = tf.reshape(y_true, [-1])
    y_pred_f = tf.reshape(y_pred, [-1])
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    score = (2. * intersection + smooth) / (tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) + smooth)
    return score

def dice_loss(y_true, y_pred):
    loss = 1 - dice_coeff(y_true, y_pred)
    return loss

def bce_dice_loss(y_true, y_pred):
    loss = losses.binary_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)
    return loss

In [43]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=bce_dice_loss,
              metrics=[dice_loss])

# model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),loss="binary_crossentropy", metrics=["acc"])
model.summary()

Model: "DoubleUNet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
conv2d_39 (Conv2D)              (None, 128, 128, 64) 1792        input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_39 (BatchNo (None, 128, 128, 64) 256         conv2d_39[0][0]                  
__________________________________________________________________________________________________
activation_39 (Activation)      (None, 128, 128, 64) 0           batch_normalization_39[0][0]     
_________________________________________________________________________________________

In [24]:
import datetime

In [25]:
save_model_path = 'C:/Users/anark/Downloads/seg_data/tmp/weights_DoubleUnet_1.hdf5'
log_dir = "C:/Users/anark/Downloads/seg_data/logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=0)
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=save_model_path, monitor='val_dice_loss', save_best_only=True, verbose=1)

In [26]:
def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * tf.math.exp(-0.1)
    
scheduler_callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [ ]:
#Step 4: Fit the u-net model
model_checkpoint = tf.keras.callbacks.ModelCheckpoint('C:/Users/anark/Downloads/seg_data/models/unet_STARE_AttentionUnet_1.hdf5', monitor='loss',verbose=1, save_best_only=True)
model.fit(X_train,y_train,epochs=50,batch_size=8,validation_data=(X_val, y_val),callbacks=[tensorboard_callback,model_checkpoint,scheduler_callback])

Epoch 1/50
102/266 [==========>...................] - ETA: 1:08 - loss: 0.7857 - dice_loss: 0.4042